In [1]:
import os
import gzip
import numpy as np
import pandas as pd
import transportation_tutorials as tt

In [2]:
# Ex1 groupby transform
districts = pd.read_csv(tt.data('FL-COUNTY-BY-DISTRICT'))
districts

,County,District
0,Charlotte,1
1,Collier,1
2,DeSoto,1
3,Glades,1
4,Hardee,1
...,...,...
62,Citrus,7
63,Hernando,7
64,Hillsborough,7
65,Pasco,7


In [3]:
bridges = pd.read_csv(tt.data('FL-BRIDGES'))
# Recall the necessary cleaning for the bridges data file
bridges = bridges.replace('-', 0)
bridges[['Poor #', 'SD #']] = bridges[['Poor #', 'SD #']].astype(int)
bridges.fillna(0, inplace=True)
bridges

,County,Total #,Good #,Fair #,Poor #,SD #,Total Area,Good Area,Fair Area,Poor Area,SD Area
0,ALACHUA (001),111,64,47,0,0,64767,55794,8973,0.0,0.0
1,BAKER (003),89,30,52,7,8,32162,19369,12282,510.0,623.0
2,BAY (005),122,49,63,10,11,210039,98834,109628,1577.0,10120.0
3,BRADFORD (007),62,23,37,2,2,9330,5492,3217,620.0,620.0
4,BREVARD (009),241,160,81,0,0,364138,204179,159959,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
63,VOLUSIA (127),227,144,76,7,7,326665,164435,147618,14612.0,14612.0
64,WAKULLA (129),50,21,23,6,6,17783,10011,7649,123.0,123.0
65,WALTON (131),219,125,86,8,9,104108,69478,34056,575.0,627.0
66,WASHINGTON (133),96,57,34,5,6,72116,46988,23224,1904.0,3220.0


In [4]:
districts['County']=districts['County'].str.lower()
bridges.iloc[:-1,0]=bridges.iloc[:-1,0].str[:-6]
bridges['County']=bridges['County'].str.lower()
# bridges.sort_values('County').to_csv('b.csv')
# districts.sort_values('County').to_csv('d.csv')

In [5]:
bridges_new=pd.merge(districts,bridges,left_on='County',right_on='County')
bridges_new.head()

,County,District,Total #,Good #,Fair #,Poor #,SD #,Total Area,Good Area,Fair Area,Poor Area,SD Area
0,charlotte,1,207,172,35,0,1,250385,229102,21284,0.0,1511.0
1,collier,1,356,281,73,2,2,190375,158006,32207,162.0,162.0
2,glades,1,59,43,16,0,0,42294,35081,7213,0.0,0.0
3,hardee,1,91,64,26,1,1,26997,20293,6548,155.0,155.0
4,hendry,1,81,63,18,0,0,18807,14827,3980,0.0,0.0


In [6]:
# Q1.Within each FDOT District, what is the fraction of structurally deficient bridge deck area in each County?
bridges_new['SD_fraction']=bridges_new.groupby('District')['SD Area'].transform(lambda x: x/x.sum())
bridges_new.head()

,County,District,Total #,Good #,Fair #,Poor #,SD #,Total Area,Good Area,Fair Area,Poor Area,SD Area,SD_fraction
0,charlotte,1,207,172,35,0,1,250385,229102,21284,0.0,1511.0,0.105179
1,collier,1,356,281,73,2,2,190375,158006,32207,162.0,162.0,0.011277
2,glades,1,59,43,16,0,0,42294,35081,7213,0.0,0.0,0.000000
3,hardee,1,91,64,26,1,1,26997,20293,6548,155.0,155.0,0.010789
4,hendry,1,81,63,18,0,0,18807,14827,3980,0.0,0.0,0.000000


In [7]:
# Q2.Which county has the highest share of structurally deficient bridge deck area within its FDOT District?
bridges_new.loc[bridges_new['SD_fraction'].idxmax(),'County']

'palm beach'

In [8]:
# Ex2 pivot tables
tour = pd.read_csv(tt.data('SERPM8-BASE2015-TOURS'))
tour.head()

,hh_id,person_id,person_num,person_type,tour_id,tour_category,tour_purpose,orig_mgra,dest_mgra,start_period,...,prob_11,prob_12,prob_13,prob_14,prob_15,prob_16,prob_17,prob_18,prob_19,prob_20
0,1690841,4502948,1,1,0,MANDATORY,Work,7736,9290,8,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1690841,4502948,1,1,1,AT_WORK,Work-Based,9290,7980,13,...,0.000319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1690841,4502948,1,1,2,AT_WORK,Work-Based,9290,10608,19,...,0.000157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1690841,4502949,2,2,0,MANDATORY,Work,7736,8289,27,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1690841,4502949,2,2,1,MANDATORY,Work,7736,8289,31,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
tour_mode_dict = {
    1: "DRIVEALONEFREE",
    2: "DRIVEALONEPAY",
    3: "SHARED2GP",
    4: "SHARED2PAY",
    5: "SHARED3GP",
    6: "SHARED3PAY",
    7: "TNCALONE",
    8: "TNCSHARED",
    9: "WALK",
    10: "BIKE",
    11: "WALK_MIX",
    12: "WALK_PRMW",
    13: "WALK_PRMD",
    14: "PNR_MIX",
    15: "PNR_PRMW",
    16: "PNR_PRMD",
    17: "KNR_MIX",
    18: "KNR_PRMW",
    19: "KNR_PRMD",
    20: "SCHBUS",
}

In [10]:
tour['tour_mode']=tour['tour_mode'].map(tour_mode_dict)
tour.head()

,hh_id,person_id,person_num,person_type,tour_id,tour_category,tour_purpose,orig_mgra,dest_mgra,start_period,...,prob_11,prob_12,prob_13,prob_14,prob_15,prob_16,prob_17,prob_18,prob_19,prob_20
0,1690841,4502948,1,1,0,MANDATORY,Work,7736,9290,8,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1690841,4502948,1,1,1,AT_WORK,Work-Based,9290,7980,13,...,0.000319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1690841,4502948,1,1,2,AT_WORK,Work-Based,9290,10608,19,...,0.000157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1690841,4502949,2,2,0,MANDATORY,Work,7736,8289,27,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1690841,4502949,2,2,1,MANDATORY,Work,7736,8289,31,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Q1.Within the Jupiter study area, what is the average distance for bike tours to work? (Hint: It is 4.03 miles)
tour_p1=tour.pivot_table(
    index='tour_purpose',
    columns='tour_mode',
    values='tour_distance'
)
print(tour_p1.loc['Work','BIKE'])
tour_p1

4.034763485477179


tour_mode,BIKE,DRIVEALONEFREE,DRIVEALONEPAY,KNR_MIX,KNR_PRMD,KNR_PRMW,PNR_MIX,PNR_PRMD,PNR_PRMW,SCHBUS,SHARED2GP,SHARED2PAY,SHARED3GP,SHARED3PAY,WALK,WALK_MIX
tour_purpose,,,,,,,,,,,,,,,,
Discretionary,2.764216,5.681107,38.526133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.469143,26.186025,7.622825,21.648273,0.637523,3.486778
Eating Out,3.739237,6.299282,21.259400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.521744,22.228364,6.677294,22.139000,0.906817,3.483111
Escort,NaN,3.848886,23.726999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.544895,23.677999,6.378542,25.274000,0.491000,5.693667
Maintenance,2.872279,4.993209,22.798727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.026752,28.893666,6.842151,20.237000,0.782691,4.597381
School,3.100353,4.396832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.816528,4.176316,NaN,4.308441,NaN,0.973146,2.434500
Shop,4.323302,8.658878,59.314263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.935834,59.844746,10.631094,59.383289,0.810000,8.946885
University,4.953091,7.974719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.809288,21.773001,8.269645,16.806000,1.343474,3.589333
Visiting,2.448940,6.533719,34.439682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.371339,33.488765,8.709989,38.686500,0.612707,3.416000
Work,4.034763,12.089045,38.741760,33.41,37.558556,39.154332,21.311667,61.115,58.980637,NaN,13.427365,39.481301,13.986657,37.353821,0.714053,9.022067


In [12]:
# Q2.What tour purpose has the highest average tour distance? (Hint: Work tours)
tour_p2=tour.pivot_table(
    index='tour_purpose',
    columns='tour_mode',
    values='tour_distance',
    margins=True
)
print(tour_p2.loc[:,'All'].idxmax())
tour_p2

Work


tour_mode,BIKE,DRIVEALONEFREE,DRIVEALONEPAY,KNR_MIX,KNR_PRMD,KNR_PRMW,PNR_MIX,PNR_PRMD,PNR_PRMW,SCHBUS,SHARED2GP,SHARED2PAY,SHARED3GP,SHARED3PAY,WALK,WALK_MIX,All
tour_purpose,,,,,,,,,,,,,,,,,
Discretionary,2.764216,5.681107,38.526133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.469143,26.186025,7.622825,21.648273,0.637523,3.486778,6.355063
Eating Out,3.739237,6.299282,21.259400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.521744,22.228364,6.677294,22.139000,0.906817,3.483111,6.341543
Escort,NaN,3.848886,23.726999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.544895,23.677999,6.378542,25.274000,0.491000,5.693667,4.266534
Maintenance,2.872279,4.993209,22.798727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.026752,28.893666,6.842151,20.237000,0.782691,4.597381,5.413463
School,3.100353,4.396832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.816528,4.176316,NaN,4.308441,NaN,0.973146,2.434500,4.534350
Shop,4.323302,8.658878,59.314263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.935834,59.844746,10.631094,59.383289,0.810000,8.946885,13.421542
University,4.953091,7.974719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.809288,21.773001,8.269645,16.806000,1.343474,3.589333,7.796431
Visiting,2.448940,6.533719,34.439682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.371339,33.488765,8.709989,38.686500,0.612707,3.416000,7.534867
Work,4.034763,12.089045,38.741760,33.41,37.558556,39.154332,21.311667,61.115,58.980637,NaN,13.427365,39.481301,13.986657,37.353821,0.714053,9.022067,15.804213


In [13]:
# Q3.What is the median distance of walking for all tour purposes? (Hint: 0.548 miles)
tour_p3=tour.pivot_table(
    columns='tour_mode',
    values='tour_distance',
    aggfunc='median'
)
print(tour_p3.loc[:,'WALK'])
tour_p3

tour_distance    0.548
Name: WALK, dtype: float64


tour_mode,BIKE,DRIVEALONEFREE,DRIVEALONEPAY,KNR_MIX,KNR_PRMD,KNR_PRMW,PNR_MIX,PNR_PRMD,PNR_PRMW,SCHBUS,SHARED2GP,SHARED2PAY,SHARED3GP,SHARED3PAY,WALK,WALK_MIX
tour_distance,2.8905,6.098,31.275,29.497999,36.278,18.084999,16.518,55.566002,56.125999,2.624,5.912,31.558,5.085,30.006001,0.548,3.294
